In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

### Create DataFrames to match tables

In [8]:
# Create the vine_table. DataFrame
from pyspark.sql.types import IntegerType
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])

vine_df = vine_df.withColumn('star_rating', vine_df['star_rating'].cast(IntegerType()))
#https://stackoverflow.com/questions/46956026/how-to-convert-column-with-string-type-to-int-form-in-pyspark-data-frame
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [10]:
#Convert vine_df to pandas df
import pandas as pd
vine_df = vine_df.toPandas()

In [12]:
vine_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,R1WBPB8MDCCN8F,5.0,0.0,0.0,N,N
1,R32M0YEWV77XG8,5.0,1.0,1.0,N,Y
2,RR8V7WR27NXJ5,1.0,0.0,0.0,N,Y
3,R1MHO5V9Z932AY,5.0,0.0,0.0,N,Y
4,R16PD71086BD2V,5.0,0.0,1.0,N,N


In [20]:
#Filter vine_df for total_votes >= 20
vine_df = vine_df.loc[vine_df['total_votes'] >= 20]
vine_df.describe()

,star_rating,helpful_votes,total_votes
count,61948.000000,61948.000000,61948.000000
mean,3.878301,51.024714,56.185284
std,1.474267,103.866780,108.907249
min,1.000000,10.000000,20.000000
25%,3.000000,21.750000,24.000000
50%,5.000000,29.000000,33.000000
75%,5.000000,48.000000,53.000000
max,5.000000,7379.000000,7512.000000


In [22]:
#Filter for >=.5 helpful_votes
vine_df = vine_df.loc[(vine_df['helpful_votes']/vine_df['total_votes'] >= .5)]
vine_df.describe()


Index(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine',
       'verified_purchase'],
      dtype='object')

In [24]:
vine_df.columns

Index(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine',
       'verified_purchase'],
      dtype='object')

In [26]:
#Split df based on vine review Y/N
vine_df_yes = vine_df.loc[(vine_df['vine'] == 'Y')]
vine_df_yes.describe()

,star_rating,helpful_votes,total_votes
count,334.000000,334.000000,334.000000
mean,4.098802,65.014970,72.368263
std,0.973739,84.583859,94.571321
min,1.000000,13.000000,20.000000
25%,4.000000,23.000000,27.000000
50%,4.000000,36.000000,41.500000
75%,5.000000,71.750000,80.750000
max,5.000000,720.000000,928.000000


In [27]:
vine_df_no = vine_df.loc[(vine_df['vine'] == 'N')]
vine_df_no.describe()

,star_rating,helpful_votes,total_votes
count,61614.000000,61614.000000,61614.000000
mean,3.877106,50.948875,56.097559
std,1.476433,103.956993,108.973941
min,1.000000,10.000000,20.000000
25%,3.000000,21.000000,24.000000
50%,5.000000,29.000000,33.000000
75%,5.000000,48.000000,53.000000
max,5.000000,7379.000000,7512.000000


In [28]:
#Find number of 5 star ratings
vine_df_yes.groupby('star_rating').count()

,review_id,helpful_votes,total_votes,vine,verified_purchase
star_rating,,,,,
1.0,8,8,8,8,8
2.0,13,13,13,13,13
3.0,56,56,56,56,56
4.0,118,118,118,118,118
5.0,139,139,139,139,139


In [29]:
vine_df_no.groupby('star_rating').count()

,review_id,helpful_votes,total_votes,vine,verified_purchase
star_rating,,,,,
1.0,9192,9192,9192,9192,9192
2.0,3592,3592,3592,3592,3592
3.0,5477,5477,5477,5477,5477
4.0,10688,10688,10688,10688,10688
5.0,32665,32665,32665,32665,32665


In [30]:
print(f'Percentage of 5 stars for vine == Y: {139/334}')
print(f'Percetage of 5 stars for vine == N: {32665/61614}')

Percentage of 5 stars for vine == Y: 0.4161676646706587
Percetage of 5 stars for vine == N: 0.5301554841432142
